# Zbx Clean Other

This notebook was created to perform semi-automatic cleaning of Zabbix database by interacting with files for mass data review and update.

This was designed to work with the hosts associated with the equipments from CELPLAM, ROHDE&SCHWARZ

For zabbix interface uses: https://github.com/lukecyca/pyzabbix

Zabbix API documentation at: https://www.zabbix.com/documentation/current/en/manual/api/reference

---

<code style="background:red;color:white"><b>ATTENTION:</b> DRAFT VERSION - PLACEHOLDER TO BE DEVELOPED.</code>

---

### Environment setup

In [1]:
URL_ZABBIX = "http://zabbixsfi.anatel.gov.br/"
TOKEN_ZABBIX = "<PASTE SUPER ADMIN TOKEN HERE>"

HOST_DATA = ["hostids", "host", "name", "status"]
INTER_DATA = ["interfaceid","hostid","type","ip","dns","main"]
GROUP_DATA = ["groupid","name"]
TEMPLATE_DATA = ["templateids","host","name"]

HOST_INTER_CSV = "zbxHostInterface.csv"
HOST_GROUP_CSV = "zbxHostGroup.csv"

CLEAN_DATA = "ZabbixClean.xlsx"
LINK_SHEET = "OutEnlace"
RFEYE_SHEET = "OutRFEye"
RMU_SHEET = "OutRMU"
OTHER_SHEET = "OutOutros"

### Includes

In [3]:
from pyzabbix import ZabbixAPI

import pandas as pd
import numpy as np

### Create session

In [3]:
zapi = ZabbixAPI(URL_ZABBIX)

zapi.session.verify = False # Disable SSL certificate verification

zapi.timeout = 5.1 #seconds

zapi.login(api_token=TOKEN_ZABBIX)

### Load host data

In [4]:
zbx_dict = zapi.host.get(output=HOST_DATA)

df_host = pd.DataFrame(zbx_dict)

### Load interface data

In [5]:
zbx_dict = zapi.hostinterface.get(output=INTER_DATA)

df_inter = pd.DataFrame(zbx_dict)

### Combine all data into a single table

Note that hosts with multiple interfaces will show up in multiple rows

In [6]:
df_full = pd.merge(df_host, df_inter, how='inner', left_on = 'hostid', right_on = 'hostid')

### Export host and interface data for external processing using CSV format

At this point one should export data from zabbix and manipulate to create a clean XLSX spreadsheet that shall be used for the new configuration

In [8]:
df_full.to_csv(HOST_INTER_CSV)

### Load template data

In [8]:
zbx_dict = zapi.template.get(output=TEMPLATE_DATA)

df_template = pd.DataFrame(zbx_dict)

In [9]:
df_template

,host,name,templateid
0,Template OS Linux by Zabbix agent,Template OS Linux by Zabbix agent,10001
1,Template App Zabbix Server,Template App Zabbix Server,10047
2,Template App Zabbix Proxy,Template App Zabbix Proxy,10048
3,Template Module Zabbix agent,Template Module Zabbix agent,10050
4,Template OS OpenBSD,Template OS OpenBSD,10074
...,...,...,...
130,Template Net MTR,Template Net MTR,10703
131,ERMx FL SmartWeb,ERMx FL SmartWeb,10744
132,Volt SmartWeb,Volt SmartWeb,10747
133,CW RMU,CW RMU,10764


### Import clean data for RFEye Stations

In [5]:
df_clean_data = pd.read_excel(CLEAN_DATA, sheet_name=RFEYE_SHEET)

In [6]:
df_clean_data

,hostid,host,name,status,interfaceid,type,ip,dns,N_host_Oficial,N_host_status,...,N_host_grp_Detentor,N_host_grp_Situação,N_host_grp_Tipo,N_host_grp_Modelo,N_host_grp_teste,N_host_grp_OVPN,IP,DNS,Série,Índice
0,10613,RFEye002073_OVPN,RFEye002073_OVPN,1,383,2,<ip address>,rfeye002073.rfeyenodes.com,False,1,...,NaN,NaN,NaN,NaN,NaN,OVPN,NaN,rfeye002073.anatel.gov.br,2073,55
1,10365,RFEye002076_OVPN,RFEye002076_OVPN,1,58,2,<ip address>,rfeye002076.rfeyenodes.com,False,1,...,NaN,NaN,NaN,NaN,NaN,OVPN,NaN,rfeye002076.anatel.gov.br,2076,56
2,10366,RFEye002079_OVPN,RFEye002079_OVPN,1,59,2,<ip address>,rfeye002079.rfeyenodes.com,False,1,...,NaN,NaN,NaN,NaN,NaN,OVPN,NaN,rfeye002079.anatel.gov.br,2079,57
3,10614,RFEye002081_OVPN,RFEye002081_OVPN,1,384,2,<ip address>,rfeye002081.rfeyenodes.com,False,1,...,NaN,NaN,NaN,NaN,NaN,OVPN,<IP Address>,rfeye002081.anatel.gov.br,2081,59
4,10496,RFEye002083_OVPN,RFEye002083_OVPN,1,189,2,<ip address>,rfeye002083.rfeyenodes.com,False,1,...,NaN,NaN,NaN,NaN,NaN,OVPN,<IP Address>,rfeye002083.anatel.gov.br,2083,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,10542,RFEye002328_OVPN,RFEye002328_OVPN,1,235,2,<ip address>,rfeye002328.rfeyenodes.com,True,1,...,GR08,Ativo,Estação,RFeye,NaN,NaN,<IP Address>,rfeye002328.anatel.gov.br,2328,239
296,10543,RFEye002330_OVPN,RFEye002330_OVPN,1,236,2,<ip address>,rfeye002330.rfeyenodes.com,True,1,...,GR01,Litígio,Estação,RFeye,NaN,NaN,NaN,rfeye002330.anatel.gov.br,2330,240
297,10673,RFEye002332,RFEye002332,0,443,2,<ip address>,rfeye002332.rfeyenodes.com,True,0,...,GR01,Ativo,Estação,RFeye,NaN,NaN,<IP Address>,rfeye002332.anatel.gov.br,2332,241
298,10545,RFEye002345,RFEye002345,0,238,2,<ip address>,NaN,True,0,...,GR05,Ativo,Estação,RFeye,NaN,NaN,<IP Address>,rfeye002345.anatel.gov.br,2345,242


# Update Group

### List all groups

In [17]:
df_clean_data.columns

Index(['hostid', 'host', 'name', 'status', 'interfaceid', 'type', 'ip', 'dns',
       'Tem OVPN?', 'Tem Teste?', 'N_host_status', 'N_host_name',
       'N_host_tag_Local', 'N_host_tag_Município', 'N_host_inv_lat',
       'N_host_inv_lon', 'N_host_inv_city', 'N_host_inv_state',
       'N_host_inv_poc_1_notes', 'N_host_inv_alias', 'N_host_inv_Asset tag',
       'N_host_inv_Tag', 'N_host_inv_Type', 'N_host_inv_Model',
       'N_host_inv_Vendor', 'N_host_grp_UF', 'N_host_grp_Detentor',
       'N_host_grp_Situação', 'N_host_grp_Tipo', 'N_host_grp_Modelo',
       'N_host_grp_teste', 'N_host_grp_OVPN', 'IP', 'DNS', 'Série', 'Índice'],
      dtype='object')

In [18]:
zbx_dict = zapi.hostgroup.get(output=GROUP_DATA)

df_group = pd.DataFrame(zbx_dict)

In [19]:
df_group

,groupid,name
0,22,AC
1,23,AL
2,21,AM
3,24,AP
4,50,Anatel Templates
...,...,...
116,110,UO113
117,69,Unidade Administrativa da Anatel
118,6,Virtual machines
119,4,Zabbix servers


### Create a dictionary with the groups to be processed

In [20]:
all_groups = {  'N_host_grp_UF':[],
                'N_host_grp_Detentor':[],
                'N_host_grp_Situação':[],
                'N_host_grp_Tipo':[],
                'N_host_grp_Modelo':[],
                'N_host_grp_teste':[],
                'N_host_grp_OVPN':[]}

for group in all_groups:
    np_array = df_clean_data[group].unique()

    np_array = np_array[~pd.isna(np_array)] # remove NaN 

    all_groups[group] = np_array.tolist()

### Create groups in Zabbix if necessary and associate all data within the dataframe with clean data

In [21]:
for group_list in all_groups:
    
    zbx_group = {'groupid':[0]*all_groups[group_list].__len__(),
                 'name':[""]*all_groups[group_list].__len__()}
    i = 0
    
    for group in all_groups[group_list]:
        
        where_is_group = df_group[df_group.isin([group])].stack().index #get the index from where the group is in the df from zabbix
        
        if len(where_is_group) == 0: #check if the group don't exist in zabbix
            
            zbx_dict = zapi.hostgroup.create(name=group)

            zbx_group['groupid'][i] = zbx_dict['groupids']
            zbx_group['name'][i] = group[0]
        else:
            index = where_is_group.to_numpy()[0][0]
            
            zbx_group['groupid'][i] = df_group['groupid'][index]
            zbx_group['name'][i] = df_group['name'][index]
            
        i = i + 1
        
    df_zbx_group = pd.DataFrame(zbx_group)
    
    df_clean_data = pd.merge(df_clean_data, df_zbx_group, how='left', left_on = group_list, right_on = 'name')
    
    df_clean_data = df_clean_data.rename(columns={"groupid": f"GID {group_list}", "name": f"zbx {group_list}"})

### Get hosts assigned to groups (BACKUP)

In [22]:
def get_group_host(zapi, df_clean_data):

    GROUP_DATA = ["groupid", "name"]
    
    df_output = pd.DataFrame(columns=['groupid', 'name', 'hostid'])

    for hostid in df_clean_data["hostid"].to_list():
        zbx_dict = zapi.hostgroup.get(output=GROUP_DATA, hostids=hostid)
        
        df = pd.DataFrame(zbx_dict)
        df['hostid']=hostid
        
        df_output = pd.concat([df_output,df],ignore_index=True)
        
    return df_output

# df_host_group = get_group_host(zapi, df_clean_data)
# df_host_group
# df_host_group.to_csv(HOST_GROUP_CSV)

### Create a list of dictionaries with groups to be removed

In [ ]:
group_dic_list=[]

df = df_group.sort_values(by=["groupid"])
group_list = df['groupid'].to_list()[1:] # get group list except the first

for group in group_list:
    group_dic_list.append({'groupid':group})

### Create a list of dictionaries with host to be edited

In [ ]:
host_dic_list=[]

host_list = df_clean_data["hostid"].to_list()

for host in host_list:
    host_dic_list.append({'hostid':host})

### Select a reference default group

In [ ]:
default_group = df['groupid'].to_list()[0]

group_List_dict = [{'groupid':default_group}]

### Add group 1 to all hosts to be edited

In [ ]:
for host in host_list:
    zbx_dict = zapi.hostgroup.massadd(groups=group_List_dict, hosts=[{'hostid':host}])

### Remove all groups from hosts:[],
except default_group

In [ ]:
for host in host_list:
    zbx_dict = zapi.hostgroup.massremove(groupids=group_list, hostids=host)

#### Add new groups from clean list

In [ ]:
def add_group(host):
    group_list = [{'groupid':host['GID N_host_grp_UF']},
                  {'groupid':host['GID N_host_grp_Detentor']},
                  {'groupid':host['GID N_host_grp_Situação']},
                  {'groupid':host['GID N_host_grp_Tipo']},
                  {'groupid':host['GID N_host_grp_Modelo']},
                  {'groupid':host['GID N_host_grp_teste']},
                  {'groupid':host['GID N_host_grp_OVPN']}]
    
    group_list = [item for item in group_list if isinstance(item['groupid'], str)]
                
    zbx_dict = zapi.hostgroup.massadd(groups=group_list, hosts=[{'hostid':host['hostid']}])
    
none = df_clean_data.apply(add_group, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
300    None
301    None
302    None
303    None
304    None
Length: 305, dtype: object

### Remove default group

In [ ]:
for host in host_list:
    zbx_dict = zapi.hostgroup.massremove(groupids=[default_group], hostids=[host])

# Update Inventory

### Remove all inventory data

In [ ]:
CLEAN_INVENTORY = { 'alias' : '',
                    'asset_tag' : '',
                    'chassis' : '',
                    'contact' : '',
                    'contract_number' : '',
                    'date_hw_decomm' : '',
                    'date_hw_expiry' : '',
                    'date_hw_install' : '',
                    'date_hw_purchase' : '',
                    'deployment_status' : '',
                    'hardware' : '',
                    'hardware_full' : '',
                    'host_netmask' : '',
                    'host_networks' : '',
                    'host_router' : '',
                    'hw_arch' : '',
                    'installer_name' : '',
                    'location' : '',
                    'location_lat' : '',
                    'location_lon' : '',
                    'macaddress_a' : '',
                    'macaddress_b' : '',
                    'model' : '',
                    'name' : '',
                    'notes' : '',
                    'oob_ip' : '',
                    'oob_netmask' : '',
                    'oob_router' : '',
                    'os' : '',
                    'os_full' : '',
                    'os_short' : '',
                    'poc_1_cell' : '',
                    'poc_1_email' : '',
                    'poc_1_name' : '',
                    'poc_1_notes' : '',
                    'poc_1_phone_a' : '',
                    'poc_1_phone_b' : '',
                    'poc_1_screen' : '',
                    'poc_2_cell' : '',
                    'poc_2_email' : '',
                    'poc_2_name' : '',
                    'poc_2_notes' : '',
                    'poc_2_phone_a' : '',
                    'poc_2_phone_b' : '',
                    'poc_2_screen' : '',
                    'serialno_a' : '',
                    'serialno_b' : '',
                    'site_address_a' : '',
                    'site_address_b' : '',
                    'site_address_c' : '',
                    'site_city' : '',
                    'site_country' : '',
                    'site_notes' : '',
                    'site_rack' : '',
                    'site_state' : '',
                    'site_zip' : '',
                    'software' : '',
                    'software_app_a' : '',
                    'software_app_b' : '',
                    'software_app_c' : '',
                    'software_app_d' : '',
                    'software_app_e' : '',
                    'software_full' : '',
                    'tag' : '',
                    'type' : '',
                    'type_full' : '',
                    'url_a' : '',
                    'url_b' : '',
                    'url_c' : '',
                    'vendor' : ''}

def set_inventory(host):    
    zbx_dict = zapi.host.update(hostid=host['hostid'], inventory_mode='0', inventory=CLEAN_INVENTORY)
    
none = df_clean_data.apply(set_inventory, axis=1)

### Add inventory data

In [ ]:
df_clean_data.columns

Index(['hostid', 'host', 'name_x', 'status', 'interfaceid', 'type', 'ip',
       'dns', 'Tem OVPN?', 'Tem Teste?', 'N_host_status', 'N_host_name',
       'N_host_tag_Local', 'N_host_tag_Município', 'N_host_inv_lat',
       'N_host_inv_lon', 'N_host_inv_city', 'N_host_inv_state',
       'N_host_inv_poc_1_notes', 'N_host_inv_alias', 'N_host_inv_Asset tag',
       'N_host_inv_Tag', 'N_host_inv_Type', 'N_host_inv_Model',
       'N_host_inv_Vendor', 'N_host_grp_UF', 'N_host_grp_Detentor',
       'N_host_grp_Situação', 'N_host_grp_Tipo', 'N_host_grp_Modelo',
       'N_host_grp_teste', 'N_host_grp_OVPN', 'IP', 'DNS', 'Série', 'Índice',
       'GID N_host_grp_UF', 'name_y', 'GID N_host_grp_Detentor',
       'zbx N_host_grp_Detentor', 'GID N_host_grp_Situação',
       'zbx N_host_grp_Situação', 'GID N_host_grp_Tipo', 'zbx N_host_grp_Tipo',
       'GID N_host_grp_Modelo', 'zbx N_host_grp_Modelo',
       'GID N_host_grp_teste', 'zbx N_host_grp_teste', 'GID N_host_grp_OVPN',
       'zbx N_host_grp

In [ ]:
host = df_clean_data.iloc[220]

inventory = {'location_lat' : f"{host['N_host_inv_lat']}",
                 'location_lon' : f"{host['N_host_inv_lon']}",
                 'site_city' :    host['N_host_inv_city'],
                 'site_state' :   host['N_host_inv_state'],
                 'poc_1_notes' :  host['N_host_inv_poc_1_notes'],
                 'alias':         host['N_host_inv_alias'],
                 'asset_tag':     host['N_host_inv_Asset tag'],
                 'tag':           host['N_host_inv_Tag'],
                 'type':          host['N_host_inv_Type'],
                 'model':         host['N_host_inv_Model'],
                 'vendor':        host['N_host_inv_Vendor']}

In [ ]:
inventory

{'location_lat': 'nan',
 'location_lon': 'nan',
 'site_city': 'Rio Largo',
 'site_state': 'AL',
 'poc_1_notes': 'GR06, UO061',
 'alias': 'Aeroporto Internacional de Maceió',
 'asset_tag': 62748.0,
 'tag': 'https://sistemas.anatel.gov.br/fiscaliza/issues/56821',
 'type': 'Estação TDOA',
 'model': 'Rfeye Node 20-6',
 'vendor': 'CRFS'}

In [ ]:
inventory = [item for item in inventory if isinstance(item['groupid'], str) or isinstance(item['groupid'], str)]

In [ ]:
def set_inventory(host):
    inventory = {'location_lat' : f"{host['N_host_inv_lat']}",
                 'location_lon' : f"{host['N_host_inv_lon']}",
                 'site_city' :    host['N_host_inv_city'],
                 'site_state' :   host['N_host_inv_state'],
                 'poc_1_notes' :  host['N_host_inv_poc_1_notes'],
                 'alias':         host['N_host_inv_alias'],
                 'asset_tag':     host['N_host_inv_Asset tag'],
                 'tag':           host['N_host_inv_Tag'],
                 'type':          host['N_host_inv_Type'],
                 'model':         host['N_host_inv_Model'],
                 'vendor':        host['N_host_inv_Vendor']}
    
    inventory = {key: value for key, value in inventory.items() if isinstance(value, str)}
    
    zbx_dict = zapi.host.update(hostid=host['hostid'], inventory_mode='0', inventory=inventory)
    
none = df_clean_data.apply(set_inventory, axis=1)

# Update Tag data

### Clean Tags

In [ ]:
def clean_tag(host):
    zbx_dict = zapi.host.update(hostid=host['hostid'], tags={})
    
none = df_clean_data.apply(clean_tag, axis=1)

### Add tag data

In [ ]:
def set_tag(host):
    tag = [{'tag' : 'Local', 'value' : host['N_host_tag_Local']},
           {'tag' : 'Município', 'value' : host['N_host_tag_Município']}]
    
    tag = [item for item in tag if isinstance(item['value'], str)]
    
    zbx_dict = zapi.host.update(hostid=host['hostid'], tags=tag)
    
none = df_clean_data.apply(set_tag, axis=1)

# Update Template

Lista hosts e template id - Not realy working... better to use the interface since there are some bug in the use of the API

In [12]:
zapi.host.get(hostid=["10396"], selectParentTemplates=["templateid","name"])

[{'hostid': '10084',
  'proxy_hostid': '0',
  'host': 'rhzbefipdin01',
  'status': '0',
  'lastaccess': '0',
  'ipmi_authtype': '-1',
  'ipmi_privilege': '2',
  'ipmi_username': '',
  'ipmi_password': '',
  'maintenanceid': '0',
  'maintenance_status': '0',
  'maintenance_type': '0',
  'maintenance_from': '0',
  'name': 'Zabbix server',
  'flags': '0',
  'templateid': '0',
  'description': '',
  'tls_connect': '1',
  'tls_accept': '1',
  'tls_issuer': '',
  'tls_subject': '',
  'proxy_address': '',
  'auto_compress': '1',
  'custom_interfaces': '0',
  'uuid': '',
  'inventory_mode': '1',
  'parentTemplates': [{'templateid': '10001',
    'name': 'Template OS Linux by Zabbix agent'},
   {'templateid': '10047', 'name': 'Template App Zabbix Server'}]},
 {'hostid': '10305',
  'proxy_hostid': '0',
  'host': 'BSA 0766753',
  'status': '0',
  'lastaccess': '0',
  'ipmi_authtype': '-1',
  'ipmi_privilege': '2',
  'ipmi_username': '',
  'ipmi_password': '',
  'maintenanceid': '0',
  'maintenance

Let us know that:
"
    {'templateid': '10308', 'name': 'RFEye Nodes'}
    {'templateid': '10764', 'name': 'CW RMU'}
"
### Select main hosts

In [13]:
df_main_host = df_clean_data[df_clean_data['N_host_Oficial']==True]



# Update host IP/DNS

In [11]:
def update_IP(host): 
    try:
        zbx_dict = zapi.hostinterface.update(interfaceid=host['interfaceid'], ip=host['IP'])
    except Exception as e:
        print(f"----> ERROR: {host['interfaceid']} - {host['IP']}")
        print(e)

df = df_clean_data[['interfaceid','IP']].copy()
df = df.dropna()
none = df.apply(update_IP, axis=1)

----> ERROR: 222 - 200146174227
('Error -32602: Invalid params., Invalid IP address "200146174227".', -32602)


In [12]:
def update_DNS(host): 
    try:
        zbx_dict = zapi.hostinterface.update(interfaceid=host['interfaceid'], dns=host['DNS'])
    except Exception as e:
        print(f"----> ERROR: {host['interfaceid']} - {host['DNS']}")
        print(e)

df = df_clean_data[['interfaceid','DNS']].copy()
df = df.dropna()
none = df.apply(update_DNS, axis=1)

# Update host names

In [15]:
def update_name(host): 
    try:
        zbx_dict = zapi.host.update(hostid=host['hostid'], host=host['N_host_name'])
    except Exception as e:
        print(f"----> ERROR: {host['hostid']} - {host['N_host_name']}")
        print(e)
        
none = df_clean_data.apply(update_name, axis=1)

----> ERROR: 10605 - RFEye002233_ovpn
('Error -32602: Invalid params., Host with the same name "RFEye002233_ovpn" already exists.', -32602)
----> ERROR: 10651 - RFEye002263_ovpn
('Error -32602: Invalid params., Host with the same name "RFEye002263_ovpn" already exists.', -32602)
----> ERROR: 10652 - RFEye002264_ovpn
('Error -32602: Invalid params., Host with the same name "RFEye002264_ovpn" already exists.', -32602)


----> ERROR: 10605 - RFEye002233_ovpn
('Error -32602: Invalid params., Host with the same name "RFEye002233_ovpn" already exists.', -32602)
----> ERROR: 10651 - RFEye002263_ovpn
('Error -32602: Invalid params., Host with the same name "RFEye002263_ovpn" already exists.', -32602)
----> ERROR: 10652 - RFEye002264_ovpn
('Error -32602: Invalid params., Host with the same name "RFEye002264_ovpn" already exists.', -32602)

# Scratch block

Output fron rfeye

{"SystemTime": "2023-04-11 06:46:10","GPS": "095353 110423 1 1 10 -2584172 -44237551 6 0 50200 ","IP": "201.73.195.82","FreeMemory": "415308 kB"}

Test VISA connection with NCAT

<code>Bash
echo -e "*IDN?\n:STATUS:TEMPERATURE?\n:GNSS:POSITION?\n*OPC\n" | nc -C CWSM211008.anatel.gov.br 37001 > teste.txt
</code>

Zabbix automation cron jobs

<code>Bash
lx.svc.fi.sensores.pd@rhzbefipdin01:~$ crontab -l

0 3 * * * sh /home/lx.svc.fi.sensores.pd/scripts_automacao/Automation_criaBD.sh

0 5 * * * sh /home/lx.svc.fi.sensores.pd/scripts_automacao/Automation_verificaBD.sh

0 6 * * * sh /home/lx.svc.fi.sensores.pd/scripts_automacao/Automation_Aplicacoes.sh
</code>

Fix hosts

In [ ]:
"""
Zabbix stores the DNS name and the IP for each host that it monitors, and
uses one or the other to connect to the host.  It is good practice to make
sure the IP and DNS name are both correct.  This script checks the DNS and
IP for all hosts in Zabbix, compares the IP against an actual DNS lookup,
and fixes it if required.
"""

import socket

from pyzabbix import ZabbixAPI, ZabbixAPIException

# The hostname at which the Zabbix web interface is available
ZABBIX_SERVER = "https://zabbix.example.com"

zapi = ZabbixAPI(ZABBIX_SERVER)

# Disable SSL certificate verification
zapi.session.verify = False

# Login to the Zabbix API
zapi.login("Admin", "zabbix")

# Loop through all hosts interfaces, getting only "main" interfaces of type "agent"
for h in zapi.hostinterface.get(
    output=["dns", "ip", "useip"], selectHosts=["host"], filter={"main": 1, "type": 1}
):
    # Make sure the hosts are named according to their FQDN
    if h["dns"] != h["hosts"][0]["host"]:
        print('Warning: {} has dns "{}"'.format(h["hosts"][0]["host"], h["dns"]))

    # Make sure they are using hostnames to connect rather than IPs (could be also filtered in the get request)
    if h["useip"] == "1":
        print("%s is using IP instead of hostname. Skipping." % h["hosts"][0]["host"])
        continue

    # Do a DNS lookup for the host's DNS name
    try:
        lookup = socket.gethostbyaddr(h["dns"])
    except socket.gaierror as e:
        print(h["dns"], e)
        continue
    actual_ip = lookup[2][0]

    # Check whether the looked-up IP matches the one stored in the host's IP
    # field
    if actual_ip != h["ip"]:
        print(
            "%s has the wrong IP: %s. Changing it to: %s"
            % (h["hosts"][0]["host"], h["ip"], actual_ip)
        )

        # Set the host's IP field to match what the DNS lookup said it should
        # be
        try:
            zapi.hostinterface.update(interfaceid=h["interfaceid"], ip=actual_ip)
        except ZabbixAPIException as e:
            print(e)